## Install Libraries


In [1]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 37.5 MB/s eta 0:00:00


## Import Libraries


In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

#kernel_size_1 = 3
#kernel_size_2 = 5
#kernel_size_3 = 7

#pad_size_1 = 2
#pad_size_2 = 2
#pad_size_3 = 3

fc_count_1 = 1024

class Net(nn.Module):
    def __init__(self, kernel_size_1, kernel_size_2, kernel_size_3, pad_size_1, pad_size_2, pad_size_3):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [6]:
def train_cifar(config):
    net = Net(
        config["kernel_size_1"], 
        config["kernel_size_2"], 
        config["kernel_size_3"], 
        config["pad_size_1"], 
        config["pad_size_2"], 
        config["pad_size_3"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-2)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["kernel_size_1"], 
        best_result.config["kernel_size_2"], 
        best_result.config["kernel_size_3"], 
        best_result.config["pad_size_1"], 
        best_result.config["pad_size_2"], 
        best_result.config["pad_size_3"]
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"kernel_size_1": tune.grid_search([3, 5, 7]), 
          "kernel_size_2": tune.grid_search([3, 5, 7]), 
          "kernel_size_3": tune.grid_search([3, 5, 7]), 
          "pad_size_1": tune.grid_search([2, 3]), 
          "pad_size_2": tune.grid_search([2, 3]), 
          "pad_size_3": tune.grid_search([2, 3])}



In [9]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


2023-04-12 03:35:03,207	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=5496) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-35-02/train_cifar_03728_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-12_03-35-04/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 68356217.92it/s] 


(train_cifar pid=5496) Extracting /root/ray_results/train_cifar_2023-04-12_03-35-02/train_cifar_03728_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-12_03-35-04/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-35-02/train_cifar_03728_00000_0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2_2023-04-12_03-35-04/data
(train_cifar pid=5496) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_03728_00000,0.7123,2023-04-12_03-36-16,True,,1cb22435115144609673ce9a7332af5c,"0_kernel_size_1=3,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.871965,172.28.0.12,5496,True,68.5348,4.54642,68.5348,1681270576,0,,10,03728_00000,0.00428724
train_cifar_03728_00001,0.727,2023-04-12_03-37-08,True,,1cb22435115144609673ce9a7332af5c,"1_kernel_size_1=5,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.811384,172.28.0.12,5496,True,51.5066,4.78216,51.5066,1681270628,0,,10,03728_00001,0.00428724
train_cifar_03728_00002,0.7127,2023-04-12_03-37-59,True,,1cb22435115144609673ce9a7332af5c,"2_kernel_size_1=7,kernel_size_2=3,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.840716,172.28.0.12,5496,True,51.4189,4.60248,51.4189,1681270679,0,,10,03728_00002,0.00428724
train_cifar_03728_00003,0.7353,2023-04-12_03-38-51,True,,1cb22435115144609673ce9a7332af5c,"3_kernel_size_1=3,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.790251,172.28.0.12,5496,True,51.2549,4.51242,51.2549,1681270731,0,,10,03728_00003,0.00428724
train_cifar_03728_00004,0.7374,2023-04-12_03-39-44,True,,1cb22435115144609673ce9a7332af5c,"4_kernel_size_1=5,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.804532,172.28.0.12,5496,True,53.0594,4.67142,53.0594,1681270784,0,,10,03728_00004,0.00428724
train_cifar_03728_00005,0.7307,2023-04-12_03-40-36,True,,1cb22435115144609673ce9a7332af5c,"5_kernel_size_1=7,kernel_size_2=5,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.792907,172.28.0.12,5496,True,51.9559,4.50562,51.9559,1681270836,0,,10,03728_00005,0.00428724
train_cifar_03728_00006,0.7114,2023-04-12_03-41-27,True,,1cb22435115144609673ce9a7332af5c,"6_kernel_size_1=3,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.907117,172.28.0.12,5496,True,51.6553,4.68056,51.6553,1681270887,0,,10,03728_00006,0.00428724
train_cifar_03728_00007,0.7349,2023-04-12_03-42-20,True,,1cb22435115144609673ce9a7332af5c,"7_kernel_size_1=5,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.82006,172.28.0.12,5496,True,52.4127,4.57233,52.4127,1681270940,0,,10,03728_00007,0.00428724
train_cifar_03728_00008,0.7086,2023-04-12_03-43-12,True,,1cb22435115144609673ce9a7332af5c,"8_kernel_size_1=7,kernel_size_2=7,kernel_size_3=3,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.894358,172.28.0.12,5496,True,52.0658,4.66023,52.0658,1681270992,0,,10,03728_00008,0.00428724
train_cifar_03728_00009,0.7322,2023-04-12_03-44-03,True,,1cb22435115144609673ce9a7332af5c,"9_kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=2,pad_size_2=2,pad_size_3=2",ef68fe3ea1a1,10,0.818004,172.28.0.12,5496,True,51.367,4.79815,51.367,1681271043,0,,10,03728_00009,0.00428724


...

(train_cifar pid=5496) Extracting /root/ray_results/train_cifar_2023-04-12_03-35-02/train_cifar_03728_00215_215_kernel_size_1=7,kernel_size_2=7,kernel_size_3=7,pad_size_1=3,pad_size_2=3,pad_size_3=3_2023-04-12_06-50-16/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_03-35-02/train_cifar_03728_00215_215_kernel_size_1=7,kernel_size_2=7,kernel_size_3=7,pad_size_1=3,pad_size_2=3,pad_size_3=3_2023-04-12_06-50-16/data
(train_cifar pid=5496) Files already downloaded and verified
(train_cifar pid=5496) Finished Training


2023-04-12 06:51:11,983	INFO tune.py:798 -- Total run time: 11767.46 seconds (11767.23 seconds for the tuning loop).


Best trial config: {'kernel_size_1': 3, 'kernel_size_2': 7, 'kernel_size_3': 5, 'pad_size_1': 2, 'pad_size_2': 3, 'pad_size_3': 3}
Best trial final validation loss: 0.7468906802356623
Best trial final validation accuracy: 0.755


100%|██████████| 170498071/170498071 [00:20<00:00, 8498993.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.7151


In [10]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
177,0.746891,0.7550,4.996705,False,56.896119,3,7,5,2,3,3
174,0.749172,0.7523,5.046355,False,54.986965,3,5,5,2,3,3
204,0.755214,0.7511,5.057071,False,59.839173,3,7,5,3,3,3
70,0.783279,0.7490,4.600187,False,47.446841,5,7,5,2,3,2
207,0.774622,0.7481,4.895053,False,51.803145,3,3,7,3,3,3
79,0.784511,0.7469,4.572402,False,47.923523,5,7,7,2,3,2
183,0.789539,0.7464,4.839243,False,49.277846,3,5,7,2,3,3
120,0.782931,0.7456,4.800349,False,56.638540,3,5,5,2,2,3
58,0.762078,0.7455,4.640300,False,51.962767,5,5,3,2,3,2
48,0.781326,0.7454,4.626278,False,43.860643,3,5,7,3,2,2


In [11]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
128,0.930914,0.7056,4.644693,False,48.362198,7,3,7,2,2,3
116,0.892410,0.7061,4.780067,False,48.310907,7,7,3,2,2,3
200,0.841078,0.7093,4.877023,False,39.890431,7,3,5,3,3,3
59,0.854421,0.7094,4.613336,False,42.688328,7,5,3,2,3,2
191,0.844853,0.7110,5.007689,False,52.984048,7,3,3,3,3,3
29,0.851968,0.7110,4.753448,False,52.231958,7,3,3,3,2,2
0,0.871965,0.7123,4.546421,False,68.534785,3,3,3,2,2,2
206,0.845426,0.7125,4.788087,False,47.702925,7,7,5,3,3,3
122,0.882651,0.7129,4.631638,False,48.608270,7,5,5,2,2,3
80,0.855178,0.7140,4.646662,False,39.640122,7,7,7,2,3,2
